In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [2]:
train = pd.read_json("train.json")

In [23]:
train["num_photos"] = train["photos"].apply(len)
train["num_features"] = train["features"].apply(len)
train["num_description_words"] = train["description"].apply(lambda x: len(x.split(" ")))
train["created"] = pd.to_datetime(train["created"])
train["created_year"] = train["created"].dt.year
train["created_month"] = train["created"].dt.month
train["created_day"] = train["created"].dt.day

In [24]:
train['features_exist'] = [0 if 0 else(1) for x in train.num_features]
train['photos_exist'] = [0 if 0 else(1) for x in train.num_photos]

In [25]:
train.head(3)

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,...,price,street_address,num_photos,num_features,num_description_words,created_year,created_month,created_day,features_exist,photos_exist
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,...,3000,792 Metropolitan Avenue,5,0,95,2016,6,24,1,1
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,...,5465,808 Columbus Avenue,11,5,9,2016,6,12,1,1
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,...,2850,241 W 13 Street,8,4,94,2016,4,17,1,1


In [161]:
num_feats = ["bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "created_month"]

num_feats2 = ["bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "created_month"]

num_feats3 = ["bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words"]

num_feats4 = ["bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words"]

num_feats5 = ["bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words", "created_month"]

In [162]:
X_df = train[num_feats]
X_mat = X_df.as_matrix()
X_mat.shape

X_df2 = train[num_feats2]
X_mat2 = X_df2.as_matrix()

X_df3 = train[num_feats3]
X_mat3 = X_df3.as_matrix()

X_df4 = train[num_feats4]
X_mat4 = X_df4.as_matrix()

X_df5 = train[num_feats5]
X_mat5 = X_df5.as_matrix()
X_mat.shape

(49352, 7)

In [163]:
Y_df = pd.DataFrame(train.loc[:,'interest_level'])
Y_df['interest'] = [0 if x=='low' else(1 if x=='medium' else(2)) for x in Y_df.interest_level]
Y_mat = Y_df.interest.as_matrix()
Y_mat.shape

(49352,)

### Random Forest Feature Selection

In [175]:
from sklearn import tree

from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import log_loss

rf_feat_select = ensemble.RandomForestClassifier(n_estimators = 1000, n_jobs = 2, min_samples_split=16, )


In [176]:
X_train, X_val, y_train, y_val = train_test_split(X_mat, Y_mat, test_size=0.3)
rf_feat_select.fit(X_train, y_train)
rf_feat_select.score(X_train, y_train)

0.84151566027904823

In [177]:
y_val_pred = rf_feat_select.predict_proba(X_val)
log_loss(y_val, y_val_pred)

0.62700506631341713

In [178]:
X_train, X_val, y_train, y_val = train_test_split(X_mat2, Y_mat, test_size=0.3)
rf_feat_select.fit(X_train, y_train)
rf_feat_select.score(X_train, y_train)

0.83844728767440513

In [179]:
y_val_pred = rf_feat_select.predict_proba(X_val)
log_loss(y_val, y_val_pred)

0.61616351351972765

In [180]:
X_train, X_val, y_train, y_val = train_test_split(X_mat3, Y_mat, test_size=0.25)
rf_feat_select.fit(X_train, y_train)
rf_feat_select.score(X_train, y_train)

0.8521100124277301

In [181]:
y_val_pred = rf_feat_select.predict_proba(X_val)
log_loss(y_val, y_val_pred)

0.62033842556106888

In [182]:
X_train, X_val, y_train, y_val = train_test_split(X_mat4, Y_mat, test_size=0.25)
rf_feat_select.fit(X_train, y_train)
rf_feat_select.score(X_train, y_train)

0.85327173501918196

In [183]:
y_val_pred = rf_feat_select.predict_proba(X_val)
log_loss(y_val, y_val_pred)

0.62457029908133121

In [184]:
X_train, X_val, y_train, y_val = train_test_split(X_mat5, Y_mat, test_size=0.25)
rf_feat_select.fit(X_train, y_train)
rf_feat_select.score(X_train, y_train)

0.85024585292051658

In [185]:
y_val_pred = rf_feat_select.predict_proba(X_val)
log_loss(y_val, y_val_pred)

0.61983762960584277

In [ ]:
from sklearn import ensemble
from sklearn.metrics import log_loss
rf_model = ensemble.RandomForestClassifier(n_estimators = 400, n_jobs = 2)

In [ ]:
import sklearn.grid_search as gs
grid_para_rf = [{'max_features':[6], 'min_samples_split': [15,16,17], 'min_samples_leaf': [1]}]

In [ ]:
grid_search_rf = gs.GridSearchCV(rf_model, grid_para_rf, scoring = 'accuracy', cv=3)

In [ ]:
grid_search_rf.fit(X_mat, Y_mat)

In [ ]:
grid_search_rf.best_estimator_

In [ ]:
grid_search_rf.best_params_

In [ ]:
grid_search_rf.best_score_

In [ ]:
grid_search_rf.grid_scores_

In [ ]:
rf_gs_model = ensemble.RandomForestClassifier(n_estimators = 1000, n_jobs = 2, \
                                               min_samples_split = 16, max_features = 6)

In [ ]:
rf_gs_model.fit(X_mat, Y_mat)

In [ ]:
rf_gs_model.score(X_mat, Y_mat)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_mat, Y_mat, test_size=0.25)

In [ ]:
rf_gs_model.fit(X_train, y_train)

In [ ]:

y_val_pred = rf_gs_model.predict_proba(X_val)

In [ ]:
log_loss(y_val, y_val_pred)

### Kaggle RF Starter Adaptation

In [ ]:
clf = ensemble.RandomForestClassifier(n_estimators=400, n_jobs=3, min_samples_split=10)
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
y_val_pred = clf.predict_proba(X_val)
log_loss(y_val, y_val_pred)

#### look for optimal samples to split by...

In [ ]:
samples_tester = []

for x in range(1, 6, 1):
    if x % 2 == 0:
        print("Checking model with %d leafmin") % (x)
    clf = ensemble.RandomForestClassifier(n_estimators=625, n_jobs=3, min_samples_split = 10, min_samples_leaf = x, oob_score=True)
    clf.fit(X_train, y_train)
    y_val_pred = clf.predict_proba(X_val)
    ll = log_loss(y_val, y_val_pred)
    samples_tester.append([x, ll, clf.oob_score_])

In [ ]:
tree = []
logloss = []
oob = []
for x in samples_tester:
    tree.append(x[0])
    logloss.append(x[1])
    oob.append(x[2])

In [ ]:
plt.scatter(tree, logloss)
plt.plot(tree, logloss)
plt.ylabel('logloss')
plt.xlabel('min sample split')
plt.title('logloss vs min sample split')
plt.show()

In [ ]:
plt.scatter(tree, oob)
plt.plot(tree, oob)
plt.ylabel('oob score')
plt.xlabel('min sample split')
plt.title('oob vs min sample split')
plt.show()

#### Search for optimal # of trees

In [ ]:
LL_tree = []

In [ ]:
for x in range(300, 1001, 100):
    if x % 200 == 0:
        print("Checking model with %d ests") % (x)
    clf = ensemble.RandomForestClassifier(n_estimators=x, n_jobs=3, oob_score=True)
    clf.fit(X_train, y_train)
    y_val_pred = clf.predict_proba(X_val)
    ll = log_loss(y_val, y_val_pred)
    LL_tree.append([x, ll, clf.oob_score_])

In [ ]:
tree = []
logloss = []
oob = []
for x in LL_tree:
    tree.append(x[0])
    logloss.append(x[1])
    oob.append(x[2])

In [ ]:
plt.scatter(tree, logloss)
plt.plot(tree, logloss)
plt.ylabel('logloss')
plt.xlabel('tree count')
plt.title('logloss vs # of trees')
plt.show()

In [ ]:
plt.scatter(tree, oob)
plt.plot(tree, oob)
plt.ylabel('oob score')
plt.xlabel('tree count')
plt.title('oob vs # of trees')
plt.show()

#### Adapt Predictions

In [186]:
test = pd.read_json(open("test.json", "r"))
print(test.shape)

(74659, 14)


In [187]:
test["num_photos"] = test["photos"].apply(len)
test["num_features"] = test["features"].apply(len)
test["num_description_words"] = test["description"].apply(lambda x: len(x.split(" ")))
test["created"] = pd.to_datetime(test["created"])
test["created_year"] = test["created"].dt.year
test["created_month"] = test["created"].dt.month
test["created_day"] = test["created"].dt.day


In [194]:
testclf = ensemble.RandomForestClassifier(n_estimators=2000, n_jobs = 2, min_samples_split = 16, max_features = 5)
#best results thus far with 1625 estimators

In [195]:
testclf.fit(X_mat, Y_mat)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=5, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=16, min_weight_fraction_leaf=0.0,
            n_estimators=2000, n_jobs=2, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [196]:
X = test[num_feats4]
y = testclf.predict_proba(X)

In [197]:
#labels2idx = {label: i for i, label in enumerate(clf.classes_)}
labels2idx = {'low': 0, 'medium': 1, 'high': 2}
labels2idx

{'high': 2, 'low': 0, 'medium': 1}

In [198]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]
sub.to_csv("submission_rf_tuned.csv", index=False)